<a href="https://colab.research.google.com/github/khush143/Multilingual-extension-of-bingliu-lexiconn/blob/main/look.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk #installing the nltk package

In [ ]:
#making the imports
import pandas as pd
import numpy as np
import multiprocessing
import re
from textblob import TextBlob
from collections import Counter
import warnings 
import seaborn as sns 
import matplotlib.pyplot as plt  

from wordcloud import WordCloud
import pickle
from nltk.stem import WordNetLemmatizer
import spacy
import nltk
nltk.download()

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import string#importing string

In [ ]:
#mounting the drive
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#importing nltk and re
import nltk
import re

In [ ]:
#importing sent_tokenize and word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

In [ ]:
#Reading the bingliu csv file
Bingliu = pd.read_csv('/content/gdrive/My Drive/NLP_ASSIGNMENT4/BingLiu.csv',sep="\t",header=None)
Bingliu.columns=["word","polarity"]
Bingliu


word  polarity
0            a+  positive
1        abound  positive
2       abounds  positive
3     abundance  positive
4      abundant  positive
...         ...       ...
6784       zaps  negative
6785     zealot  negative
6786    zealous  negative
6787  zealously  negative
6788     zombie  negative

[6789 rows x 2 columns]

In [ ]:
#opening the english.txt file and  reading it
englishcorp = open("english.txt", "r")
english=englishcorp.read()

In [ ]:
#displaying the content of the english.txt file read
english

'Judging from previous posts this used to be a good place , but not any longer .\nWe , there were four of us , arrived at noon - the place was empty - and the staff acted like we were imposing on them and they were very rude .\nThey never brought us complimentary noodles , ignored repeated requests for sugar , and threw our dishes on the table .\nThe food was lousy - too sweet or too salty and the portions tiny .\nAfter all that , they complained to me about the small tip .\nAvoid this place !\nI have eaten at Saul , many times , the food is always consistently , outrageously good .\nSaul is the best restaurant on Smith Street and in Brooklyn .\nThe duck confit is always amazing and the foie gras terrine with figs was out of this world .\nThe wine list is interesting and has many good values .\nFor the price , you can not eat this well in Manhattan .\nI was very disappointed with this restaurant .\nIve asked a cart attendant for a lotus leaf wrapped rice and she replied back rice and j

In [ ]:
#reading the english_hindi-dictionary
df2 = pd.read_csv('english-hindi-dictionary.txt', sep=" ", header=None, names=["eng","pro", "hindi"])

In [ ]:
df3=df2[["eng","hindi"]]
df3

eng            hindi
0                   the                द
1                     .                ।
2                     ,                ,
3                     i              मैं
4                    to             करने
...                 ...              ...
142746            zx-5n            zx-5n
142747          zx-6000          zx-6000
142748           zx5000           zx5000
142749  zzzzzzzzzzzzzzz  zzzzzzzzzzzzzzz
142750            <unk>            <unk>

[142751 rows x 2 columns]

In [ ]:
L1=pd.DataFrame(columns=["english_word","hindi_word","polarity"])#creating a dataframe L1 with columns provided
#L1.columns=["english word","hindi word","polarity"]
L1

Empty DataFrame
Columns: [english_word, hindi_word, polarity]
Index: []

In [ ]:
for i in range(0,len(Bingliu)):#lowering the words present in the bingliu
  Bingliu.loc[i,"word"].lower()

In [ ]:
for j in range(0,len(df3)):#lowering the words present in df3
  df3.loc[i,"eng"].lower()

In [ ]:
count=0#This is just for checking purpose to see how any words of bingliu are there in the dictionary
for i in range(0,len(Bingliu)):
  for j in range(0,len(df3)):
    if((Bingliu.loc[i,"word"])==(df3.loc[j,"eng"])):
      count=count+1
print(count)

In [ ]:
#This step creates L1.
for i in range(0,len(Bingliu)):
  for j in range(0,len(df3)):
    if((Bingliu.loc[i,"word"])==(df3.loc[j,"eng"])):
      #L1["english word"]=Bingliu.loc[i,"word"]
      #L1["hindi word"]=df3.loc[j,"hindi"]
      #L1["polarity"]=Bingliu.loc[i,"polarity"]
      new_row = {'english_word':Bingliu.loc[i,"word"], 'hindi_word':df3.loc[j,"hindi"], 'polarity':Bingliu.loc[i,"polarity"]}
     
      L1 = L1.append(new_row, ignore_index=True)#APPENDING ROWS TO THE DATAFRAME




In [ ]:
#THIS function is for preprocessing in the hindi corpus
def textPreHin(text):
    no_punct = "".join([i for i in text if i not in string.punctuation]) #removing punctuation
    no_numbers="".join([i for i in no_punct if i not in "0123456789"])
    hindi_actual=re.sub(r'[a-zA-z0-9 ]+'," ",no_numbers)
    
    tokens = word_tokenize(hindi_actual) 
    return(tokens)

In [ ]:
#reading the hindi.txt file
hin_corpus = pd.read_csv (r'/content/gdrive/My Drive/NLP_ASSIGNMENT4/hindi.txt', sep = "\n",encoding="utf8",header = None)
hin_corpus.columns = ['sentences']
hin_corpus['clean text'] = hin_corpus['sentences'].apply(lambda t: textPreHin(t))#calling thr textPreHin
df = hin_corpus['clean text']
hin_vect = df.values.tolist()#converting dataframe to list

In [ ]:
hin_vect[1:5]

[['इस',
  'पॉपअप',
  'बॉक्स',
  'में',
  'पासवर्ड',
  'चेंज',
  'करने',
  'लॉगइन',
  'अलर्ट्स',
  'चालू',
  'करने',
  'और',
  'मौजूदा',
  'फेसबुक',
  'सेशन',
  'को',
  'पूरी',
  'तरह',
  'से',
  'बंद',
  'करने',
  'के',
  'ऑप्शन',
  'मिलेंगे',
  '।'],
 ['अब',
  'नए',
  'फीचर',
  'से',
  'इस',
  'तरह',
  'की',
  'गलतियां',
  'नहीं',
  'होंगी',
  'और',
  'अकाउंट',
  'को',
  'हैक',
  'करना',
  'बेहद',
  'मुश्किल',
  'हो',
  'जाएगा',
  '।'],
 ['एंड्रॉयड',
  'किटकैट',
  'या',
  'उससे',
  'ऊपर',
  'के',
  'वर्जन',
  'पर',
  'काम',
  'करता',
  'है',
  '।'],
 ['पेरिस्कोप',
  'एप',
  'में',
  'रिज्यूम',
  'नोटिफिकेशन',
  'फीचर',
  'दिया',
  'गया',
  'है',
  'जो',
  'बहुत',
  'ही',
  'खास',
  'है',
  '।']]

In [ ]:
#FUNCTION FOR REMOVING THE PUNCTUATIONS
def punctuations(text):
  
    regex_pattern = re.compile(r'[\,+,\:\?\!\"\(\)|!\'\\%\[\]]+')
    clean_text = regex_pattern.sub(r' ', text)
    clean_text = clean_text.replace('-', '')
    return clean_text
m1=punctuations(english)


In [ ]:
m1

'Judging from previous posts this used to be a good place   but not any longer .\nWe   there were four of us   arrived at noon  the place was empty  and the staff acted like we were imposing on them and they were very rude .\nThey never brought us complimentary noodles   ignored repeated requests for sugar   and threw our dishes on the table .\nThe food was lousy  too sweet or too salty and the portions tiny .\nAfter all that   they complained to me about the small tip .\nAvoid this place  \nI have eaten at Saul   many times   the food is always consistently   outrageously good .\nSaul is the best restaurant on Smith Street and in Brooklyn .\nThe duck confit is always amazing and the foie gras terrine with figs was out of this world .\nThe wine list is interesting and has many good values .\nFor the price   you can not eat this well in Manhattan .\nI was very disappointed with this restaurant .\nIve asked a cart attendant for a lotus leaf wrapped rice and she replied back rice and just

In [ ]:
#function removing the alphanumerics
def remove_alphanumerics(text):
   
    txt = []
    for each in text.split():
        if not any(x in each.lower() for x in "0123456789"):
            txt.append(each)
    txtsent = " ".join(txt)
    return txtsent 
z=remove_alphanumerics(m1)


In [ ]:
z

'Judging from previous posts this used to be a good place but not any longer . We there were four of us arrived at noon the place was empty and the staff acted like we were imposing on them and they were very rude . They never brought us complimentary noodles ignored repeated requests for sugar and threw our dishes on the table . The food was lousy too sweet or too salty and the portions tiny . After all that they complained to me about the small tip . Avoid this place I have eaten at Saul many times the food is always consistently outrageously good . Saul is the best restaurant on Smith Street and in Brooklyn . The duck confit is always amazing and the foie gras terrine with figs was out of this world . The wine list is interesting and has many good values . For the price you can not eat this well in Manhattan . I was very disappointed with this restaurant . Ive asked a cart attendant for a lotus leaf wrapped rice and she replied back rice and just walked away . I had to ask her three

In [ ]:
f1=remove_alphanumerics(f2)

In [ ]:
#for removing any english words from the hindi corpus
hindi2=re.sub(r'[a-zA-z0-9 ]+'," ",f1)
hindi2

'फेसबुक का सिक्योरिटी चैकअप फीचर पॉपअप की तरह यूजर्स को दिखाइ देगा । इस पॉपअप बॉक्स में पासवर्ड चेंज करने लॉगइन अलर्ट्स चालू करने और मौजूदा फेसबुक सेशन को पूरी तरह से बंद करने के ऑप्शन मिलेंगे । अब नए फीचर से इस तरह की गलतियां नहीं होंगी और अकाउंट को हैक करना बेहद मुश्किल हो जाएगा । एंड्रॉयड किटकैट या उससे ऊपर के वर्जन पर काम करता है । पेरिस्कोप एप में रिज्यूम नोटिफिकेशन फीचर दिया गया है जो बहुत ही खास है । इस फीचर के तहत यूजर ब्रॉडकास्ट के जरिए देखे जाने वाले वीडियो रूकने पर फिर वहीं से देख सकते हैं जहां वो रूका था । इस फीचर के तहत ब्रॉडकास्टर का टाइम और मोबाइल डेटा दोनों की बचत होती है । आपके स्मार्टफोन में ज्यादातर एप ऎसे ही हैं जिनसे आपको अपनी जेब का पैसा खर्च कर ही कुछ काम लिया जा सकता है लेकिन एक ऎसी एप भी है जिसमें आपको ना केवल फ्री में रिचार्ज मिल सकता है बल्कि डिस्काउंट कूपन भी प्राप्त किए जा सकते हैं । पर मैसेज डिलीवरी व्हाट्सप्प और आर्इमैसेज की तुलना में बहुत तेज है । व्हाट्सप्प जैसा फीचर है । जिस भी व्यक्ति ने एक ओएस के रूप में ब्लैकबेरी का अनुभव नहीं किया है इस पर बीबीएम क

In [ ]:
#lowering the case
final_english=z.lower()

In [ ]:
#printing the text after preprocessing
final_english

'judging from previous posts this used to be a good place but not any longer . we there were four of us arrived at noon the place was empty and the staff acted like we were imposing on them and they were very rude . they never brought us complimentary noodles ignored repeated requests for sugar and threw our dishes on the table . the food was lousy too sweet or too salty and the portions tiny . after all that they complained to me about the small tip . avoid this place i have eaten at saul many times the food is always consistently outrageously good . saul is the best restaurant on smith street and in brooklyn . the duck confit is always amazing and the foie gras terrine with figs was out of this world . the wine list is interesting and has many good values . for the price you can not eat this well in manhattan . i was very disappointed with this restaurant . ive asked a cart attendant for a lotus leaf wrapped rice and she replied back rice and just walked away . i had to ask her three

In [ ]:
#importing stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#removing the stopwords
def stop_words(text):
    
    #stop_words = set(stop_words)
    split_list = text.split(" ")
    split_list = [word for word in split_list if word not in stop]
    return " ".join(split_list)
final_english=stop_words(l)

In [ ]:
#printing the text after removing the stopwords
final_english

'judging from previous posts this used to be a good place but not any longer . we there were four of us arrived at noon the place was empty and the staff acted like we were imposing on them and they were very rude . they never brought us complimentary noodles ignored repeated requests for sugar and threw our dishes on the table . the food was lousy too sweet or too salty and the portions tiny . after all that they complained to me about the small tip . avoid this place i have eaten at saul many times the food is always consistently outrageously good . saul is the best restaurant on smith street and in brooklyn . the duck confit is always amazing and the foie gras terrine with figs was out of this world . the wine list is interesting and has many good values . for the price you can not eat this well in manhattan . i was very disappointed with this restaurant . ive asked a cart attendant for a lotus leaf wrapped rice and she replied back rice and just walked away . i had to ask her three

In [ ]:
#tokenizing into sentences
sents=sent_tokenize(final_english)
sents

In [ ]:
#replacing dots 
for i in range(0,len(sents)):
  re.sub('.',"",str(i))

In [ ]:
sents

In [ ]:

sents1=re.sub(r'\.',"",str(sents))

In [ ]:
sents1

"['judging from previous posts this used to be a good place but not any longer ', 'we there were four of us arrived at noon the place was empty and the staff acted like we were imposing on them and they were very rude ', 'they never brought us complimentary noodles ignored repeated requests for sugar and threw our dishes on the table ', 'the food was lousy too sweet or too salty and the portions tiny ', 'after all that they complained to me about the small tip ', 'avoid this place i have eaten at saul many times the food is always consistently outrageously good ', 'saul is the best restaurant on smith street and in brooklyn ', 'the duck confit is always amazing and the foie gras terrine with figs was out of this world ', 'the wine list is interesting and has many good values ', 'for the price you can not eat this well in manhattan ', 'i was very disappointed with this restaurant ', 'ive asked a cart attendant for a lotus leaf wrapped rice and she replied back rice and just walked away 

In [ ]:
#creating the word list
words_in_list=[]
for s in sents:

  words=word_tokenize(s)
  words_in_list.append(words)
words_in_list  


In [ ]:
#replacing the dots in words_in_list
sents2=re.sub(r'\.',"",str(words_in_list))


In [ ]:
sents2

"[['judging', 'from', 'previous', 'posts', 'this', 'used', 'to', 'be', 'a', 'good', 'place', 'but', 'not', 'any', 'longer', ''], ['we', 'there', 'were', 'four', 'of', 'us', 'arrived', 'at', 'noon', 'the', 'place', 'was', 'empty', 'and', 'the', 'staff', 'acted', 'like', 'we', 'were', 'imposing', 'on', 'them', 'and', 'they', 'were', 'very', 'rude', ''], ['they', 'never', 'brought', 'us', 'complimentary', 'noodles', 'ignored', 'repeated', 'requests', 'for', 'sugar', 'and', 'threw', 'our', 'dishes', 'on', 'the', 'table', ''], ['the', 'food', 'was', 'lousy', 'too', 'sweet', 'or', 'too', 'salty', 'and', 'the', 'portions', 'tiny', ''], ['after', 'all', 'that', 'they', 'complained', 'to', 'me', 'about', 'the', 'small', 'tip', ''], ['avoid', 'this', 'place', 'i', 'have', 'eaten', 'at', 'saul', 'many', 'times', 'the', 'food', 'is', 'always', 'consistently', 'outrageously', 'good', ''], ['saul', 'is', 'the', 'best', 'restaurant', 'on', 'smith', 'street', 'and', 'in', 'brooklyn', ''], ['the', 'duc

In [ ]:
words=word_tokenize(final_english)
words

In [ ]:
words=[word_tokenize(w) for w in sents]
print(words)

In [ ]:
final_english1=re.sub(r'\.',"",final_english)

In [ ]:
words1=word_tokenize(final_english1)

In [ ]:
words1

In [ ]:
!pip install gensim

In [ ]:
import gensim #importing gensim and Word2Vec
from gensim.models import Word2Vec 

In [ ]:
actuale = gensim.models.Word2Vec(size=150, min_count=1, workers=4)#training the model for english text file
actuale.build_vocab(words_in_list)
actuale.train(words_in_list, total_examples=len(words_in_list), epochs=400)

(20799572, 29723200)

In [ ]:
actualh = gensim.models.Word2Vec(size=150, min_count=1, workers=4)#training the model for hindi text file
actualh.build_vocab(hin_vect)
actualh.train(hin_vect, total_examples=len(hin_vect), epochs=400)

(27646795, 37681600)

In [ ]:
actualh.save("Actuallyh.model")#saving the model

In [ ]:
actuale.save("Actuallye.model")#saving the model

In [ ]:
vocabe=list(actuale.wv.vocab)#finding out the vocab for english corpus

In [ ]:
len(vocabe)#finding out the length of vocabulory

6070

In [ ]:
vocabh=list(actualh.wv.vocab)#finding out the vocab for hindi corpus

In [ ]:
len(vocabh)

9250

In [ ]:
actuale.wv.vectors#finding out the vectors

array([[ 0.88081795,  0.27126455, -1.9439324 , ...,  0.53884006,
        -0.0176786 , -0.28982762],
       [ 1.7246131 ,  0.6969001 , -0.6973589 , ..., -0.35884902,
        -0.16412504, -0.10028429],
       [ 0.07052246, -0.9597375 , -1.4231261 , ...,  0.71135557,
         0.49529526, -0.28203246],
       ...,
       [ 0.33332187,  0.5814891 ,  1.097602  , ...,  0.5808776 ,
         2.1489587 ,  0.9616404 ],
       [ 0.8362212 , -0.00230616,  0.1864013 , ..., -0.00817429,
         1.2290337 , -0.20241956],
       [ 0.20706056, -0.11738686, -0.7304585 , ..., -1.6350286 ,
         1.3159928 , -0.786145  ]], dtype=float32)

In [ ]:
actualh.wv.most_similar("रोमांचक",topn=5)#finding out similar words

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('कथ्य', 0.3039725422859192),
 ('जेम्स', 0.2996327877044678),
 ('आयाम', 0.28240644931793213),
 ('कहानी', 0.2800702154636383),
 ('शौकीन', 0.27015605568885803)]

In [ ]:
actuale.wv.most_similar("wine",topn=5)#finding out the similar words for wine

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('extensive', 0.3002370595932007),
 ('sake', 0.2933518588542938),
 ('waste', 0.27523696422576904),
 ('dry', 0.26779428124427795),
 ('presentation', 0.2601240873336792)]

In [ ]:
#loading the hindi english lexicon created
L2= pd.read_csv('/content/gdrive/My Drive/NLP_ASSIGNMENT4/L2.csv')

In [ ]:
L2

Unnamed: 0    english_word hindi_word  polarity
0              0        abundant      हुस्न  positive
1              1      accessible       सुलभ  positive
2              2      accomplish       पूरा  positive
3              3    accomplished       पूरा  positive
4              4  accomplishment    उपलब्धि  positive
...          ...             ...        ...       ...
2240        2240        wrinkles  झुर्रियों  negative
2241        2241           wrong        गलत  negative
2242        2242         wrongly        गलत  negative
2243        2243             zap       ज़ैप  negative
2244        2244          zombie     ज़ोंबी  negative

[2245 rows x 4 columns]

In [ ]:
L3=L2[["english_word","hindi_word","polarity"]]

In [ ]:
L3


english_word hindi_word  polarity
0           abundant      हुस्न  positive
1         accessible       सुलभ  positive
2         accomplish       पूरा  positive
3       accomplished       पूरा  positive
4     accomplishment    उपलब्धि  positive
...              ...        ...       ...
2240        wrinkles  झुर्रियों  negative
2241           wrong        गलत  negative
2242         wrongly        गलत  negative
2243             zap       ज़ैप  negative
2244          zombie     ज़ोंबी  negative

[2245 rows x 3 columns]

In [ ]:
#creating a list of english_words in L3
words_in_L3=[]
for i in range(0,len(L3)):
  word_in_L3=L3.loc[i,"english_word"]
  words_in_L3.append(word_in_L3)
words_in_L3

['abundant',
 'accessible',
 'accomplish',
 'accomplished',
 'accomplishment',
 'accomplishments',
 'accurate',
 'accurately',
 'achievement',
 'achievements',
 'acumen',
 'adjustable',
 'admirable',
 'abundant',
 'accessible',
 'accomplish',
 'accomplished',
 'accomplishment',
 'accomplishments',
 'accurate',
 'accurately',
 'achievement',
 'achievements',
 'acumen',
 'adjustable',
 'admirable',
 'admire',
 'admirer',
 'adorable',
 'adore',
 'advanced',
 'advantage',
 'advantages',
 'advocates',
 'affection',
 'affinity',
 'affirmative',
 'afford',
 'affordable',
 'agile',
 'agility',
 'all-around',
 'alluring',
 'amaze',
 'amazed',
 'amazing',
 'amazingly',
 'ambitious',
 'ample',
 'angel',
 'angelic',
 'appeal',
 'appealing',
 'appreciate',
 'appreciated',
 'appropriate',
 'approval',
 'approve',
 'aspire',
 'assurance',
 'assurances',
 'assuring',
 'astonished',
 'astonishing',
 'astounding',
 'attentive',
 'attraction',
 'attractive',
 'attractively',
 'auspicious',
 'authentic',


In [ ]:
len(words_in_L3)

2245

In [ ]:
#ENGLISH WORDS IN THE DICTIONARY 
eng_word_in_dictionary=[]
for i in range(0,len(df3)):
  eng_word=df3.loc[i,"eng"]
  eng_word_in_dictionary.append(eng_word)
len(eng_word_in_dictionary)


142751

In [ ]:
#HINDI WORD IN THE DICTIONARY
hindi_word_in_dictionary=[]
for i in range(0,len(df3)):
  hindi_word=df3.loc[i,"hindi"]
  hindi_word_in_dictionary.append(hindi_word)
hindi_word_in_dictionary


['द',
 '।',
 ',',
 'मैं',
 'करने',
 'और',
 'एक',
 'यह',
 'है',
 'की',
 'यह',
 'के',
 'वह',
 'में',
 'आप',
 'के',
 'मेरा',
 'पर',
 'पास',
 'लेकिन',
 'नहीं',
 'था',
 '&apos;t',
 ')',
 'के',
 '(',
 '!',
 '&apos;s',
 'हैं',
 'तो',
 'होना',
 'कर',
 'एक',
 'अगर',
 'बहुत',
 'वे',
 'पर',
 '&quot;',
 'या',
 'महान',
 'सभी',
 'उपयोग',
 'से',
 'था',
 'अच्छा',
 'बस',
 'जब',
 'एक',
 'ऊपर',
 'बाहर',
 'अपने',
 'होगा',
 'है',
 'कोई',
 'पाएं',
 'करेंगे',
 'की',
 'मुझे',
 'के',
 'कैमरा',
 'केवल',
 'इन',
 'अधिक',
 'की',
 '/',
 'उन्हें',
 'गुणवत्ता',
 'ध्वनि',
 'उत्पाद',
 'अच्छी',
 'अन्य',
 'वहाँ',
 'कौन',
 ':',
 'क्या',
 'समय',
 'क्या',
 'इसके',
 'खरीदा',
 'मूल्य',
 'डॉन',
 'कुछ',
 'के',
 '-',
 'यहां',
 'काम',
 '.',
 'काम',
 'हूँ',
 'ज्यादा',
 'किसी',
 '&apos;ve',
 'वास्तव',
 'क्योंकि',
 'खरीदें',
 'द्वारा',
 'अब',
 '$',
 'आसान',
 '&apos;m',
 'बेहतर',
 'कम',
 'इस्तेमाल',
 'प्लेयर',
 'करता',
 'छोटा',
 'हम',
 'किया',
 'तब',
 'का',
 'इकाई',
 'पहला',
 'इसकी',
 'मिला',
 'अभी',
 'नया',
 'वापस',
 'हो',
 'की',
 '

In [ ]:
len(eng_word_in_corpus)

69544

In [ ]:
hindi_word_in_corpus=hindi_words

In [ ]:
len(hindi_word_in_corpus)

93731

In [ ]:
#Function for finding most similar word corresponding to english_word and hindi_word
def find_most_similar(english_word,hindi_word,polarity):
   most_english=actuale.wv.most_similar(english_word,topn=5)
   most_hindi=actualh.wv.most_similar(hindi_word,topn=5)
   return most_english,most_hindi

In [ ]:
for i in range(0,len(L3)):#checking if the words in L3 belong to the vocab
  if((L3.loc[i,"english_word"] in vocabe) and (L3.loc[i,"hindi_word"] in vocabh)):
    english_word=L3.loc[i,"english_word"]
    hindi_word=L3.loc[i,"hindi_word"]
    polarity=L3.loc[i,"polarity"]
    eng,hindi=find_most_similar(english_word,hindi_word,polarity)#calling the function most_similar
    #print(i)
    #print(eng)
    #print(hindi)
    similar_hindi=[]
    similar_english=[]
    for similar1 in eng:
       similar_english.append(similar1[0])
    for similar2 in hindi:
       similar_hindi.append(similar2[0])
    #print(similar_english)
    #print(similar_hindi)
    Append_to_L2(similar_english,similar_hindi,polarity,L3)#Calling the function Append_to_L2

      

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
function for finding the matched pairs
def Append_to_L2(english_similar,hindi_similar,polarity,L3):
  for i in range(0,len(english_similar)):
    for j in range(0,len(hindi_similar)):
      for k in range(0,len(df3)):
        if((df3.loc[k,"eng"]==english_similar[i]) and (df3.loc[k,"hindi"]==hindi_similar[j])):
          new_row = {'english_word':english_similar[i], 'hindi_word':hindi_similar[j], 'polarity':polarity}
          L3= L3.append(new_row, ignore_index=True)



In [ ]:
len(L3)

2245